In [5]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
from scipy.io.wavfile import write

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from sem_data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from sem_models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

def load_data_from_json(file_path):
    with open(file_path, 'r') as file:
        test_text_sem_dic = json.load(file)
    for key in test_text_sem_dic:
        test_text_sem_dic[key] = torch.tensor(test_text_sem_dic[key])
    return test_text_sem_dic

def ensure_directory_exists(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

## 单一话者

In [6]:
# choose pretrained model and the trained step
model = 'ljs_sem_mat_phone'
# model = 'ljs_sem_mat_text'
# model = 'ljs_sem_mat_bert_phone'
# model = 'ljs_sem_mat_bert_text'
# model = 'onehour_ljs_sem_mat_phone'
# model = 'onehour_ljs_sem_mat_text'

# model = 'librif_sem_mat_phone'
# model = 'librif_sem_mat_text'
# model = 'librif_sem_mat_bert_phone'
# model = 'librif_sem_mat_bert_text'

# model = 'emovdb_sem_mat_phone'
# model = 'emovdb_sem_mat_text'
# model = 'emovdb_sem_mat_bert_phone'
# model = 'emovdb_sem_mat_bert_text'

# step = 'G_50000'
step = "G_100000"
# step = "G_150000"
# step = "G_200000"
# step = "G_250000"
# step = "G_300000"


common_dir = '/data/vitsGPT/vits/'
log_dir = f'{common_dir}sem_vits/logs/'
save_dir = f'{log_dir}{model}/{step}/source_model_test_wav'
ensure_directory_exists(save_dir)
hps = utils.get_hparams_from_file(f"{log_dir}{model}/config.json")
sem_embedding = hps.data.sem_embedding
print(f"configs: {sem_embedding}")

# Dictionary to map the model to its corresponding test_text_sem_dic_file
model_to_test_text_sem_dic = {
    'ljs_sem_mat_phone': 'ljs_text_sem_mat_phone_t5120.json',
    'ljs_sem_mat_text': 'ljs_text_sem_mat_text_t5120.json',
    'ljs_sem_mat_bert_text': 'ljs_text_bert_text_768.json',
    'ljs_sem_mat_bert_phone': 'ljs_text_bert_phone_768.json',
    'onehour_ljs_sem_mat_phone': 'ljs_text_sem_mat_phone_t5120.json',
    'onehour_ljs_sem_mat_text': 'ljs_text_sem_mat_text_t5120.json',
    'librif_sem_mat_phone': 'librif_text_sem_mat_phone_t5120.json',
    'librif_sem_mat_text': 'librif_text_sem_mat_text_t5120.json',
    'librif_sem_mat_bert_text': 'librif_text_bert_text_768.json',
    'librif_sem_mat_bert_phone': 'librif_text_bert_phone_768.json',
    'emovdb_sem_mat_phone': 'emovdb_text_sem_mat_phone_t5120.json',
    'emovdb_sem_mat_text': 'emovdb_text_sem_mat_text_t5120.json',
    'emovdb_sem_mat_bert_text': 'emovdb_text_bert_text_768.json',
    'emovdb_sem_mat_bert_phone': 'emovdb_text_bert_phone_768.json',
}
# Get the corresponding test_text_sem_dic_file for the chosen model
test_text_sem_dic_file = model_to_test_text_sem_dic[model]
test_text_sem_dic = load_data_from_json(f"{common_dir}filelists/{test_text_sem_dic_file}")

configs: /data/vitsGPT/vits/filelists/ljs_audio_sem_mat_phone_t5120.pt


In [7]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()
        
_ = utils.load_checkpoint(f"{log_dir}{model}/{step}.pth", net_g, None) 

In [8]:
keys = list(test_text_sem_dic.keys())
range_limit = len(keys)
range_limit = min(range_limit, len(keys))
print(f"range_limit: {range_limit}")
m = 5

for i in range(range_limit):
    key = keys[i]  # key 是字符串
    s = get_text(key, hps)  # 将 key 赋值给 s
    print(key)
    with torch.no_grad():
        x_tst = s.cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([s.size(0)]).cuda()     
        emb_sem = test_text_sem_dic[key].cuda().unsqueeze(0)  # 从字典中获取对应的张量
        emb_sem_lengths = torch.LongTensor([emb_sem.size(1)]).cuda()    
        print(f"x_tst: {x_tst.shape}")
        print(f"x_tst_lengths: {x_tst_lengths}")
        print(f"emb_sem: {emb_sem.shape}")
        print(f"emb_sem_lengths: {emb_sem_lengths}")
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1, emb_sem=emb_sem, emb_sem_lengths=emb_sem_lengths)[0][0,0].data.cpu().float().numpy()
    if i < m:
        ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
    write(f"{save_dir}/output_sem_{i}.wav", hps.data.sampling_rate, audio)


range_limit: 500
Mrs. De Mohrenschildt thought that Oswald,
x_tst: torch.Size([1, 89])
x_tst_lengths: tensor([89], device='cuda:0')
emb_sem: torch.Size([1, 39, 5120])
emb_sem_lengths: tensor([39], device='cuda:0')


The Secret Service believed that it was very doubtful that any President would ride regularly in a vehicle with a fixed top, even though transparent.
x_tst: torch.Size([1, 311])
x_tst_lengths: tensor([311], device='cuda:0')
emb_sem: torch.Size([1, 137, 5120])
emb_sem_lengths: tensor([137], device='cuda:0')


Between the hours of eight and nine p.m. they were occupied with the children in the bedrooms located at the extreme east end of the house.
x_tst: torch.Size([1, 297])
x_tst_lengths: tensor([297], device='cuda:0')
emb_sem: torch.Size([1, 137, 5120])
emb_sem_lengths: tensor([137], device='cuda:0')


The prisoner had nothing to deal with but wooden panels, and by dint of cutting and chopping he got both the lower panels out.
x_tst: torch.Size([1, 263])
x_tst_lengths: tensor([263], device='cuda:0')
emb_sem: torch.Size([1, 113, 5120])
emb_sem_lengths: tensor([113], device='cuda:0')


Under these circumstances, unnatural as they are, with proper management, the bean will thrust forth its radicle and its plumule;
x_tst: torch.Size([1, 273])
x_tst_lengths: tensor([273], device='cuda:0')
emb_sem: torch.Size([1, 123, 5120])
emb_sem_lengths: tensor([123], device='cuda:0')


Oswald demonstrated his thinking in connection with his return to the United States by preparing two sets of identical questions of the type which he might have thought
x_tst: torch.Size([1, 355])
x_tst_lengths: tensor([355], device='cuda:0')
emb_sem: torch.Size([1, 153, 5120])
emb_sem_lengths: tensor([153], device='cuda:0')
it is not possible to state with scientific certainty that a particular small group of fibers come from a certain piece of clothing
x_tst: torch.Size([1, 275])
x_tst_lengths: tensor([275], device='cuda:0')
emb_sem: torch.Size([1, 123, 5120])
emb_sem_lengths: tensor([123], device='cuda:0')
has confidence in the dedicated Secret Service men who are ready to lay down their lives for him
x_tst: torch.Size([1, 211])
x_tst_lengths: tensor([211], device='cuda:0')
emb_sem: torch.Size([1, 97, 5120])
emb_sem_lengths: tensor([97], device='cuda:0')
Since these agencies are already obliged constantly to evaluate the activities of such groups,
x_tst: torch.Size([1, 211])
x_tst_l